In [1]:
#https://www.analyticsvidhya.com/blog/2020/06/nlp-project-information-extraction/
    
from urllib.request import urlopen
from random import seed, randint
from nltk.tokenize.punkt import PunktSentenceTokenizer, PunktParameters
from src.preprocessing import PreProcessing
import os
from pathlib import Path
import pandas as pd
import codecs

In [2]:
# seed random number generator
seed(16)
BASE_DIR = Path(os.path.abspath('')).resolve()
objPreProcessing = PreProcessing()

In [3]:
def processSenteces(id_doc_base,text,category) -> list:

    punkt_params = PunktParameters()
    punkt_params.abbrev_types = set(['Mr', 'Mrs', 'LLC','art', 'lei','nr'])
    tokenizer = PunktSentenceTokenizer(punkt_params)
    tokens = tokenizer.tokenize(text)

    sentenceList = []
    ordem = 0
    for sentence in tokens:
        ordem = ordem + 1
        id_doc_base = id_doc_base
        if(len(sentence)  > 100):
            nr_tokens = objPreProcessing.getNrTokens(sentence)
            id_al = randint(1,9999)
            idSentenca = f'{id_doc_base}_SENT_{id_al}'
            sentenceList.append({"id_sentenca": idSentenca
                                 ,"ordem":ordem
                                 , "id_doc_base": id_doc_base
                                 ,"tokens_total":nr_tokens["tokens_total"]
                                 ,"tokens_no_stopwords":nr_tokens["tokens_no_stopwords"]
                                 ,"cat_sentenca":category
                                 ,"text_sentenca":sentence
                             })
    return sentenceList


### ProcessText

#### Insert DocBase

In [4]:
## Saving dataset
dfBaseLegis = pd.read_json(Path(os.path.join(BASE_DIR,'dataset', 'legislacao.json')))
dfBaseLegis.to_parquet(Path(os.path.join(BASE_DIR,'dataset', 'processed','legislacao_base.parquet')), engine='fastparquet')

In [5]:
dfBaseLegis.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35 entries, 0 to 34
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   path         35 non-null     object
 1   id_doc_base  35 non-null     object
 2   name         35 non-null     object
 3   esfera       35 non-null     object
 4   cat          35 non-null     object
 5   ementa       35 non-null     object
dtypes: object(6)
memory usage: 1.8+ KB


### Load text end sentences

#### Building the "third sector" dataset for expert evaluation


In [8]:
dfSentenca = pd.DataFrame()

for index,row in dfBaseLegis.iterrows():

    data_location = Path(os.path.join(BASE_DIR,'dataset','html',f'{row["path"]}'))
    text = objPreProcessing.getTextFromHTML(data_location)
    text = objPreProcessing.clearText(text[150:len(text)-300])
    sentenceList = processSenteces(row["id_doc_base"],text,row["cat"]) 

    if row['cat'] == 'terceiro_setor':
        output_dict = [x['text_sentenca'] for x in sentenceList if x['cat_sentenca'] == 'terceiro_setor']

        if(len(output_dict)) > 0:
            with open(Path(os.path.join(BASE_DIR,'dataset','sentences','to_evaluate',f'{row["path"]}.txt')), 'w', encoding='utf-8') as infile:
                for line in output_dict:
                    infile.write(line)
                    infile.write('\n \n')
    else:
       dfSentenca = pd.concat([dfSentenca,pd.DataFrame(sentenceList)])


In [9]:
dfSentenca.head(2)

,id_sentenca,ordem,id_doc_base,tokens_total,tokens_no_stopwords,cat_sentenca,text_sentenca
0,L14133_SENT_5952,2,L14133,105,65,licitacoes,O PRESIDENTE DA REPÚBLICA FAÇO SABER QUE O CON...
1,L14133_SENT_903,3,L14133,74,45,licitacoes,1 NÃO SÃO ABRANGIDAS POR ESTA LEI AS EMPRESAS ...


In [10]:
dfSentenca.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6097 entries, 0 to 725
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   id_sentenca          6097 non-null   object
 1   ordem                6097 non-null   int64 
 2   id_doc_base          6097 non-null   object
 3   tokens_total         6097 non-null   int64 
 4   tokens_no_stopwords  6097 non-null   int64 
 5   cat_sentenca         6097 non-null   object
 6   text_sentenca        6097 non-null   object
dtypes: int64(3), object(4)
memory usage: 381.1+ KB


#### Getting sentences evaluated by experts

In [11]:
pathFilesAlProc = Path(os.path.join(BASE_DIR,'dataset','sentences','already_evaluated'))
for root, subdirs, files in os.walk(pathFilesAlProc):
    for filename in files:
        id = filename.split('.')[0]
        with codecs.open(Path(os.path.join(pathFilesAlProc,filename)), encoding='utf-8') as f:
                sentences = [line.strip() for line in f.readlines() if line.strip() != '']

        for sentence in sentences:
            sentenceList = processSenteces(id,sentence,"terceiro_setor") 
            dfSentenca = pd.concat([dfSentenca,pd.DataFrame(sentenceList)])



In [12]:
dfBaseLegis = pd.merge(dfBaseLegis,dfSentenca,on="id_doc_base", how="left")

In [13]:
dfBaseLegis

,path,id_doc_base,name,esfera,cat,ementa,id_sentenca,ordem,tokens_total,tokens_no_stopwords,cat_sentenca,text_sentenca
0,L13019.htm,L1309,"LEI 13.019, DE 31 DE JULHO DE 2014",federal,terceiro_setor,Estabelece o regime jurídico das parcerias ent...,NaN,NaN,NaN,NaN,NaN,NaN
1,L104062002_TER_SET.htm,L104062002_TER_SET,"LEI N 10.406, DE 10 DE JANEIRO DE 2002",federal,terceiro_setor,Institui o Código Civil.,L104062002_TER_SET_SENT_7717,1.0,81.0,44.0,terceiro_setor,"Sob pena de nulidade, o estatuto das associaçõ..."
2,L104062002_TER_SET.htm,L104062002_TER_SET,"LEI N 10.406, DE 10 DE JANEIRO DE 2002",federal,terceiro_setor,Institui o Código Civil.,L104062002_TER_SET_SENT_2227,1.0,15.0,11.0,terceiro_setor,"Os associados devem ter iguais direitos, mas o..."
3,L104062002_TER_SET.htm,L104062002_TER_SET,"LEI N 10.406, DE 10 DE JANEIRO DE 2002",federal,terceiro_setor,Institui o Código Civil.,L104062002_TER_SET_SENT_9545,2.0,38.0,21.0,terceiro_setor,Se o associado for titular de quota ou fração ...
4,L104062002_TER_SET.htm,L104062002_TER_SET,"LEI N 10.406, DE 10 DE JANEIRO DE 2002",federal,terceiro_setor,Institui o Código Civil.,L104062002_TER_SET_SENT_6225,1.0,27.0,16.0,terceiro_setor,A exclusão do associado só é admissível havend...
...,...,...,...,...,...,...,...,...,...,...,...,...
7177,DEL2848compilado.htm,DEL2848,"DECRETO-LEI No 2.848, DE 7 DE DEZEMBRO DE 1940",federal,codigo_penal,Código Penal.,DEL2848_SENT_3782,919.0,56.0,37.0,codigo_penal,CAPÍTULO III DOS CRIMES CONTRA O FUNCIONAMENTO...
7178,DEL2848compilado.htm,DEL2848,"DECRETO-LEI No 2.848, DE 7 DE DEZEMBRO DE 1940",federal,codigo_penal,Código Penal.,DEL2848_SENT_9175,920.0,55.0,38.0,codigo_penal,ART. 359 O. . VIOLÊNCIA POLÍTICA ART. 359 P. R...
7179,DEL2848compilado.htm,DEL2848,"DECRETO-LEI No 2.848, DE 7 DE DEZEMBRO DE 1940",federal,codigo_penal,Código Penal.,DEL2848_SENT_5566,921.0,49.0,35.0,codigo_penal,ART. 359 Q. . CAPÍTULO IV DOS CRIMES CONTRA O ...
7180,DEL2848compilado.htm,DEL2848,"DECRETO-LEI No 2.848, DE 7 DE DEZEMBRO DE 1940",federal,codigo_penal,Código Penal.,DEL2848_SENT_8103,922.0,54.0,36.0,codigo_penal,CAPÍTULO V CAPÍTULO VI DISPOSIÇÕES COMUNS ART....


In [14]:
dfBaseLegis[dfBaseLegis["cat"]=='sociedade_anomima'].head(2)

,path,id_doc_base,name,esfera,cat,ementa,id_sentenca,ordem,tokens_total,tokens_no_stopwords,cat_sentenca,text_sentenca
4942,L6404consol.htm,L6404,"LEI N 6.404, DE 15 DE DEZEMBRO DE 1976",federal,sociedade_anomima,Dispõe sobre as Sociedades por Ações.,L6404_SENT_3830,2.0,59.0,35.0,sociedade_anomima,"O PRESIDENTE DA REPÚBLICA, FAÇO SABER QUE O CO..."
4943,L6404consol.htm,L6404,"LEI N 6.404, DE 15 DE DEZEMBRO DE 1976",federal,sociedade_anomima,Dispõe sobre as Sociedades por Ações.,L6404_SENT_4719,3.0,25.0,17.0,sociedade_anomima,OBJETO SOCIAL ART. 2 PODE SER OBJETO DA COMPAN...


In [15]:
## Saving dataset
dfBaseLegis.to_parquet(Path(os.path.join(BASE_DIR,'dataset', 'processed','legislacao_base_sentencas.parquet')), engine='fastparquet')